# CNN vs MLP

In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, \
GlobalAveragePooling2D, Input, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
import numpy as np
import os

Using TensorFlow backend.
/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/t

### Definitions

In [2]:
def read_data_original():
    #READ DATA FRAME
    (full_x_train, full_y_train), (full_x_test, full_y_test) = cifar10.load_data()
    full_x_train = full_x_train.astype('float32')
    
    #z-score
    mean = np.mean(full_x_train,axis=(0,1,2,3))
    std = np.std(full_x_train,axis=(0,1,2,3))
    full_x_train = (full_x_train-mean)/(std+1e-7)
    full_x_test = (full_x_test-mean)/(std+1e-7)
    
    return (full_x_train, full_y_train), (full_x_test, full_y_test)

### Load data

In [3]:
(full_x_train, full_y_train), (full_x_test, full_y_test) = read_data_original()

num_classes = 10

full_y_train = np_utils.to_categorical(full_y_train,num_classes)
full_y_test = np_utils.to_categorical(full_y_test,num_classes)

##### Define data generator

In [4]:
datagen = ImageDataGenerator(
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=5,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=5,
        channel_shift_range=0.1,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        validation_split=0.0)

In [5]:
datagen.fit(full_x_train)

### Build model

In [6]:
model = Sequential()

model.add(Conv2D(100, kernel_size=3, padding="same", activation="relu", input_shape=(32,32,3)))
model.add(Conv2D(100, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))

model.add(Conv2D(200, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(200, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))


model.add(Conv2D(400, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(400, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))

model.add(Conv2D(800, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(800, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(GlobalAveragePooling2D())


model.add(Dropout(0.125))
model.add(Dense(2000))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax", name="predictions"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 100)       2800      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 100)       90100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 100)       400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 100)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 200)       180200    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 200)       360200    
__________

#### Training

In [8]:
model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.SGD(lr=0.01),
            metrics=['accuracy'])

In [9]:
batch_size = 200
epochs = 15

In [10]:
callbacks = []

In [11]:
model.fit_generator(datagen.flow(full_x_train, full_y_train, batch_size=batch_size),
                        steps_per_epoch=len(full_x_train)//batch_size,
                        validation_data=(full_x_test, full_y_test),
                        epochs=epochs, verbose=1, workers=20,
                        callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
250/250 [==============================] - 67s 267ms/step - loss: 1.6912 - acc: 0.4160 - val_loss: 1.3533 - val_acc: 0.5121
Epoch 2/15
250/250 [==============================] - 58s 233ms/step - loss: 1.2180 - acc: 0.5642 - val_loss: 1.4801 - val_acc: 0.5224
Epoch 3/15
250/250 [==============================] - 61s 242ms/step - loss: 1.0465 - acc: 0.6272 - val_loss: 0.9544 - val_acc: 0.6660
Epoch 4/15
250/250 [==============================] - 61s 244ms/step - loss: 0.9304 - acc: 0.6718 - val_loss: 0.9007 - val_acc: 0.6871
Epoch 5/15
250/250 [==============================] - 61s 242ms/step - loss: 0.8511 - acc: 0.7011 - val_loss: 0.8138 - val_acc: 0.7169
Epoch 6/15
250/250 [==============================] - 61s 244ms/step - loss: 0.7880 - acc: 0.7231 - val_loss: 0.8020 - val_acc: 0.7224
Epoch 7/15
250/250 [==============================] - 61s 243ms/step - loss: 0.7313 - acc: 0.7441 - val_loss: 0.8275 - val_acc: 0.7231
Epoch 8

In [12]:
scores = model.evaluate(full_x_test, full_y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 1s 116us/step


In [13]:
print(scores)

[0.5345783334970474, 0.8167000019550323]


### Dense model

In [14]:
model_dense = Sequential()
#model_dense.add(Input(shape=(32,32,3)))
model_dense.add(Flatten(input_shape=(32,32,3)))


model_dense.add(Dropout(0.125))
model_dense.add(Dense(2000))
model_dense.add(Dropout(0.25))
model_dense.add(Dense(10, activation="softmax", name="predictions"))

In [15]:
model_dense.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 3072)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              6146000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 2000)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                20010     
Total params: 6,166,010
Trainable params: 6,166,010
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_dense.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.SGD(lr=0.001),
            metrics=['accuracy'])

In [17]:
model_dense.fit_generator(datagen.flow(full_x_train, full_y_train, batch_size=batch_size),
                        steps_per_epoch=len(full_x_train)//batch_size,
                        validation_data=(full_x_test, full_y_test),
                        epochs=epochs, verbose=1, workers=20,
                        callbacks=callbacks)

Epoch 1/15
250/250 [==============================] - 63s 252ms/step - loss: 2.6071 - acc: 0.2063 - val_loss: 1.9805 - val_acc: 0.3125
Epoch 2/15
250/250 [==============================] - 59s 237ms/step - loss: 2.4136 - acc: 0.2391 - val_loss: 1.9000 - val_acc: 0.3419
Epoch 3/15
250/250 [==============================] - 58s 234ms/step - loss: 2.3413 - acc: 0.2525 - val_loss: 1.8631 - val_acc: 0.3537
Epoch 4/15
250/250 [==============================] - 61s 242ms/step - loss: 2.3008 - acc: 0.2583 - val_loss: 1.8423 - val_acc: 0.3581
Epoch 5/15
250/250 [==============================] - 72s 287ms/step - loss: 2.2607 - acc: 0.2663 - val_loss: 1.8255 - val_acc: 0.3675
Epoch 6/15
250/250 [==============================] - 71s 283ms/step - loss: 2.2378 - acc: 0.2696 - val_loss: 1.8094 - val_acc: 0.3687
Epoch 7/15
250/250 [==============================] - 70s 279ms/step - loss: 2.2172 - acc: 0.2753 - val_loss: 1.8042 - val_acc: 0.3744
Epoch 8/15
250/250 [==============================] - 6

In [18]:
scores_dense = model_dense.evaluate(full_x_test, full_y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 0s 49us/step


In [19]:
print(scores_dense)

[1.75955637216568, 0.39199999868869784]


### Conclusions

For images and video processing, advantages of CNN over MLP constitute in:
- ability to extract spacial features (in a "natural" way, with a sliding window)
- shift robustness
- lower complexity (for larger images)
- tranferrability to different image size